# 📊 H1/H2 Eyeball Test - Variable Distribution & Relationships

이 notebook은 H1/H2 모델의 모든 변수들을 시각적으로 확인합니다.

## Models

**H1: Early Funding (OLS)**
```
early_funding_musd ~ z_vagueness + z_employees_log + founder_serial +
                     is_hardware + z_firm_age + C(sector_fe) + C(founding_cohort)
```

**H2: Growth (Logit)**
```
growth ~ z_vagueness * is_hardware + founder_serial +
         z_employees_log + C(founding_cohort)
```

## Variables

**종속변수 (Dependent Variables)**
- `early_funding_musd`: Early stage funding amount in millions USD (H1)
- `growth`: Binary indicator for Series B+ achievement (H2)

**핵심 독립변수 (Key Independent Variables)**
- `z_vagueness`: Strategic vagueness score (z-scored)
- `is_hardware`: Architecture type (0=Software, 1=Hardware)

**통제변수 (Control Variables)**
- `z_employees_log`: Employee count (log-transformed, z-scored)
- `z_firm_age`: Firm age in years (z-scored)
- `founder_serial`: Serial entrepreneur indicator (0=First-time, 1=Serial)
- `founding_cohort`: Founding year cohort (categorical)
- `sector_fe`: Sector fixed effects (H1 only)

---

## Setup

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Configure display
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

print("✅ Setup complete!")

✅ Setup complete!


---
## 1. Load Data

파이프라인 Step 2에서 생성된 features를 로드합니다.

**필수 사전 작업:**
```bash
# 1. Features 생성 (캐시가 있으면 10초, 없으면 7분)
python -m src.cli engineer-features
```

In [4]:
%%time
# Load features dataset
data_file = Path("/Users/hyunjimoon/tolzul/Front/On/love(cs)/strategic_ambiguity/empirics/data/processed/features_all.parquet")

if not data_file.exists():
    print(f"❌ Data not found: {data_file}")
    print(f"\n   Run pipeline first:")
    print(f"   python -m src.cli engineer-features")
    raise FileNotFoundError(f"Missing: {data_file}")

print(f"📂 Loading data from {data_file}")
df = pd.read_parquet(data_file)

print(f"\n✅ Data loaded successfully!")
print(f"   Companies: {len(df):,}")
print(f"   Features:  {len(df.columns)}")
print(f"   Memory:    {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

📂 Loading data from /Users/hyunjimoon/tolzul/Front/On/love(cs)/strategic_ambiguity/empirics/data/processed/features_all.parquet


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

---
## 2. Summary Statistics

모든 변수들의 기술통계를 확인합니다.

In [3]:
# Continuous variables summary
cont_vars = ['z_vagueness', 'early_funding_musd', 'z_employees_log', 'z_firm_age']

print("="*80)
print("CONTINUOUS VARIABLES - Summary Statistics")
print("="*80)

summary_data = []
for var in cont_vars:
    if var in df.columns:
        data = df[var].dropna()
        summary_data.append({
            'Variable': var,
            'Count': f"{len(data):,}",
            'Mean': f"{data.mean():.4f}",
            'Std': f"{data.std():.4f}",
            'Min': f"{data.min():.4f}",
            '25%': f"{data.quantile(0.25):.4f}",
            '50%': f"{data.median():.4f}",
            '75%': f"{data.quantile(0.75):.4f}",
            'Max': f"{data.max():.4f}"
        })
    else:
        print(f"\n⚠️  {var}: NOT AVAILABLE")

summary_df = pd.DataFrame(summary_data)
display(summary_df)

print("\n💡 What to look for:")
print("   - z_vagueness: Should have mean ≈ 0, std ≈ 1 (z-scored)")
print("   - early_funding_musd: Should be positive, typically $1M-$50M range")
print("   - z_employees_log: Should have mean ≈ 0, std ≈ 1 (z-scored)")
print("   - z_firm_age: Should have mean ≈ 0, std ≈ 1 (z-scored)")

CONTINUOUS VARIABLES - Summary Statistics


NameError: name 'df' is not defined

In [ ]:
# Categorical variables summary
cat_vars = ['growth', 'is_hardware', 'founder_serial']

print("="*80)
print("CATEGORICAL VARIABLES - Frequency Distribution")
print("="*80)

for var in cat_vars:
    if var in df.columns:
        print(f"\n{var}:")
        counts = df[var].value_counts().sort_index()
        
        freq_data = []
        for idx, cnt in counts.items():
            pct = cnt / len(df) * 100
            freq_data.append({
                'Value': idx,
                'Count': f"{cnt:,}",
                'Percentage': f"{pct:.2f}%"
            })
        
        freq_df = pd.DataFrame(freq_data)
        display(freq_df)
    else:
        print(f"\n⚠️  {var}: NOT AVAILABLE")

print("\n💡 What to look for:")
print("   - growth: Should be 20-40% (typical Series B+ progression rate)")
print("   - is_hardware: Should have both values for interaction test")
print("   - founder_serial: May be all 0 if founder data unavailable")

---
## 3. Univariate Distributions

각 변수의 분포를 시각화합니다.

In [ ]:
# Plot continuous variable distributions
continuous_vars = [
    ('z_vagueness', 'Strategic Vagueness (z-scored)', 'green'),
    ('early_funding_musd', 'Early Funding ($M)', 'red'),
    ('z_employees_log', 'Employee Count (log, z-scored)', 'orange'),
    ('z_firm_age', 'Firm Age (z-scored)', 'purple'),
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, (var, label, color) in enumerate(continuous_vars):
    ax = axes[i]
    
    if var not in df.columns:
        ax.text(0.5, 0.5, f'{var}\nNot Available',
               ha='center', va='center', fontsize=14, color='red')
        ax.set_title(label, fontweight='bold')
        continue
    
    data = df[var].dropna()
    
    # Histogram
    ax.hist(data, bins=50, color=color, alpha=0.7, edgecolor='black')
    
    # Add mean and median lines
    mean_val = data.mean()
    median_val = data.median()
    
    ax.axvline(mean_val, color='darkred', linestyle='--', linewidth=2,
              label=f'Mean: {mean_val:.2f}')
    ax.axvline(median_val, color='orange', linestyle=':', linewidth=2,
              label=f'Median: {median_val:.2f}')
    
    ax.set_xlabel(label, fontsize=11, fontweight='bold')
    ax.set_ylabel('Count', fontsize=11)
    ax.set_title(f'{label}\n(μ={mean_val:.2f}, σ={data.std():.2f})',
                fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ Continuous variable distributions plotted")

In [ ]:
# Plot categorical variable distributions
categorical_vars = [
    ('growth', 'Growth to Series B+', ['gray', 'blue']),
    ('is_hardware', 'Architecture Type', ['skyblue', 'gray']),
    ('founder_serial', 'Founder Type', ['lightcoral', 'darkred']),
]

fig, axes = plt.subplots(1, 3, figsize=(14, 5))

for i, (var, label, colors) in enumerate(categorical_vars):
    ax = axes[i]
    
    if var not in df.columns:
        ax.text(0.5, 0.5, f'{var}\nNot Available',
               ha='center', va='center', fontsize=14, color='red')
        ax.set_title(label, fontweight='bold')
        continue
    
    counts = df[var].value_counts().sort_index()
    
    ax.bar(counts.index, counts.values, color=colors, alpha=0.7, edgecolor='black')
    
    # Add value labels
    for idx, val in counts.items():
        pct = val / len(df) * 100
        ax.text(idx, val, f'{val:,}\n({pct:.1f}%)',
               ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Labels
    if var == 'growth':
        ax.set_xticklabels(['No Growth', 'Series B+'])
    elif var == 'is_hardware':
        ax.set_xticklabels(['Software', 'Hardware'])
    elif var == 'founder_serial':
        ax.set_xticklabels(['First-time', 'Serial'])
    
    ax.set_xlabel(label, fontsize=11, fontweight='bold')
    ax.set_ylabel('Count', fontsize=11)
    ax.set_title(label, fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n✅ Categorical variable distributions plotted")

---
## 4. Bivariate Relationships - H1 Model

**H1 모델**: `early_funding_musd ~ z_vagueness + z_employees_log + founder_serial + is_hardware + z_firm_age`

종속변수(Early Funding)와 각 독립변수들의 관계를 확인합니다.

In [ ]:
# H1: Early Funding vs Vagueness
if 'early_funding_musd' in df.columns and 'z_vagueness' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    sample_df = df[['early_funding_musd', 'z_vagueness']].dropna()
    if len(sample_df) > 10000:
        sample_df = sample_df.sample(10000, random_state=42)
    
    ax.scatter(sample_df['z_vagueness'], sample_df['early_funding_musd'],
              alpha=0.3, s=20, color='green')
    
    # Add trend line
    z = np.polyfit(sample_df['z_vagueness'], sample_df['early_funding_musd'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(sample_df['z_vagueness'].min(),
                        sample_df['z_vagueness'].max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=2,
           label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
    
    ax.set_xlabel('Vagueness (z-scored)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Early Funding ($M)', fontsize=12, fontweight='bold')
    ax.set_title('H1: Early Funding ~ Vagueness', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Correlation
    corr = sample_df['z_vagueness'].corr(sample_df['early_funding_musd'])
    print(f"\n📊 Correlation: {corr:.4f}")
    if abs(corr) < 0.05:
        print(f"   ⚠️  Very weak correlation")
    elif abs(corr) < 0.15:
        print(f"   Weak correlation")
    else:
        print(f"   Moderate correlation")
else:
    print("❌ Cannot plot - variables not available")

In [ ]:
# H1: Early Funding by Hardware
if 'early_funding_musd' in df.columns and 'is_hardware' in df.columns:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    hw_df = df[['early_funding_musd', 'is_hardware']].dropna()
    hw_labels = ['Software', 'Hardware']
    
    data_to_plot = [
        hw_df[hw_df['is_hardware']==0]['early_funding_musd'],
        hw_df[hw_df['is_hardware']==1]['early_funding_musd']
    ]
    
    bp = ax.boxplot(data_to_plot, labels=hw_labels, patch_artist=True)
    bp['boxes'][0].set_facecolor('skyblue')
    bp['boxes'][1].set_facecolor('gray')
    
    ax.set_ylabel('Early Funding ($M)', fontsize=12, fontweight='bold')
    ax.set_title('H1: Early Funding by Architecture', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics by group
    print("\n📊 Early Funding by Architecture:")
    print(hw_df.groupby('is_hardware')['early_funding_musd'].describe())
else:
    print("❌ Cannot plot - variables not available")

In [ ]:
# H1: Early Funding vs Other Controls
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Early Funding vs Employees
ax = axes[0]
if 'early_funding_musd' in df.columns and 'z_employees_log' in df.columns:
    sample_df = df[['early_funding_musd', 'z_employees_log']].dropna()
    if len(sample_df) > 10000:
        sample_df = sample_df.sample(10000, random_state=42)
    
    ax.scatter(sample_df['z_employees_log'], sample_df['early_funding_musd'],
              alpha=0.3, s=20, color='orange')
    
    z = np.polyfit(sample_df['z_employees_log'], sample_df['early_funding_musd'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(sample_df['z_employees_log'].min(),
                        sample_df['z_employees_log'].max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=2,
           label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
    
    ax.set_xlabel('Employees (log, z-scored)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Early Funding ($M)', fontsize=11, fontweight='bold')
    ax.set_title('H1: Early Funding ~ Employees', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'Data not available', ha='center', va='center')

# 2. Early Funding vs Firm Age
ax = axes[1]
if 'early_funding_musd' in df.columns and 'z_firm_age' in df.columns:
    sample_df = df[['early_funding_musd', 'z_firm_age']].dropna()
    if len(sample_df) > 10000:
        sample_df = sample_df.sample(10000, random_state=42)
    
    ax.scatter(sample_df['z_firm_age'], sample_df['early_funding_musd'],
              alpha=0.3, s=20, color='purple')
    
    z = np.polyfit(sample_df['z_firm_age'], sample_df['early_funding_musd'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(sample_df['z_firm_age'].min(),
                        sample_df['z_firm_age'].max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=2,
           label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
    
    ax.set_xlabel('Firm Age (z-scored)', fontsize=11, fontweight='bold')
    ax.set_ylabel('Early Funding ($M)', fontsize=11, fontweight='bold')
    ax.set_title('H1: Early Funding ~ Firm Age', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
else:
    ax.text(0.5, 0.5, 'Data not available', ha='center', va='center')

plt.tight_layout()
plt.show()

print("\n✅ H1 bivariate relationships plotted")

---
## 5. Bivariate Relationships - H2 Model

**H2 모델**: `growth ~ z_vagueness * is_hardware + founder_serial + z_employees_log`

종속변수(Growth)와 각 독립변수들의 관계를 확인합니다.

In [ ]:
# H2: Growth rate by Vagueness quintile
if 'growth' in df.columns and 'z_vagueness' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df_h2 = df[['growth', 'z_vagueness']].dropna()
    # Use duplicates='drop' to handle duplicate bin edges
    df_h2['v_quintile'] = pd.qcut(df_h2['z_vagueness'], q=5,
                                   labels=False, duplicates='drop')
    # Create labels based on actual number of bins
    n_bins = df_h2['v_quintile'].nunique()
    bin_labels = [f'Q{i+1}' for i in range(n_bins)]
    df_h2['v_quintile'] = df_h2['v_quintile'].map(dict(enumerate(bin_labels)))
    
    growth_by_v = df_h2.groupby('v_quintile')['growth'].agg(['mean', 'count'])
    
    ax.bar(range(len(growth_by_v)), growth_by_v['mean'], color='green', alpha=0.7, edgecolor='black')
    ax.set_xticks(range(len(growth_by_v)))
    ax.set_xticklabels(growth_by_v.index)
    ax.set_xlabel('Vagueness Quintile', fontsize=12, fontweight='bold')
    ax.set_ylabel('Growth Rate (Pr[Series B+])', fontsize=12, fontweight='bold')
    ax.set_title(f'H2: Growth ~ Vagueness (by {n_bins} bins)', fontsize=14, fontweight='bold')
    
    # Add value labels
    for i, (val, cnt) in enumerate(zip(growth_by_v['mean'], growth_by_v['count'])):
        ax.text(i, val, f'{val:.2%}\n(n={int(cnt):,})',
               ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, growth_by_v['mean'].max() * 1.2)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Growth Rate by Vagueness Quintile:")
    print(growth_by_v)
else:
    print("❌ Cannot plot - variables not available")

In [ ]:
# H2: Growth rate by Hardware
if 'growth' in df.columns and 'is_hardware' in df.columns:
    fig, ax = plt.subplots(figsize=(8, 6))
    
    growth_by_hw = df.groupby('is_hardware')['growth'].agg(['mean', 'count'])
    
    colors = ['skyblue', 'gray']
    ax.bar(range(len(growth_by_hw)), growth_by_hw['mean'],
          color=colors, alpha=0.7, edgecolor='black')
    ax.set_xticks(range(len(growth_by_hw)))
    ax.set_xticklabels(['Software', 'Hardware'])
    ax.set_ylabel('Growth Rate (Pr[Series B+])', fontsize=12, fontweight='bold')
    ax.set_title('H2: Growth by Architecture', fontsize=14, fontweight='bold')
    
    for i, (val, cnt) in enumerate(zip(growth_by_hw['mean'], growth_by_hw['count'])):
        ax.text(i, val, f'{val:.2%}\n(n={int(cnt):,})',
               ha='center', va='bottom', fontweight='bold', fontsize=11)
    
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, growth_by_hw['mean'].max() * 1.2)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Growth Rate by Architecture:")
    print(growth_by_hw)
else:
    print("❌ Cannot plot - variables not available")

In [ ]:
# H2: Vagueness × Hardware Interaction (MONEY PLOT!)
if 'growth' in df.columns and 'z_vagueness' in df.columns and 'is_hardware' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df_interact = df[['growth', 'z_vagueness', 'is_hardware']].dropna()
    # Use duplicates='drop' to handle duplicate bin edges
    df_interact['v_quintile'] = pd.qcut(df_interact['z_vagueness'], q=5,
                                        labels=False, duplicates='drop')
    # Create labels based on actual number of bins
    n_bins_interact = df_interact['v_quintile'].nunique()
    bin_labels_interact = [f'Q{i+1}' for i in range(n_bins_interact)]
    df_interact['v_quintile'] = df_interact['v_quintile'].map(dict(enumerate(bin_labels_interact)))
    
    growth_sw = df_interact[df_interact['is_hardware']==0].groupby('v_quintile')['growth'].mean()
    growth_hw = df_interact[df_interact['is_hardware']==1].groupby('v_quintile')['growth'].mean()
    
    x = range(len(growth_sw))
    ax.plot(x, growth_sw, marker='o', linewidth=3, markersize=10,
           color='skyblue', label='Software', zorder=3)
    ax.plot(x, growth_hw, marker='s', linewidth=3, markersize=10,
           color='gray', linestyle='--', label='Hardware', zorder=3)
    
    ax.set_xticks(x)
    ax.set_xticklabels(growth_sw.index)
    ax.set_xlabel('Vagueness Quintile', fontsize=12, fontweight='bold')
    ax.set_ylabel('Growth Rate (Pr[Series B+])', fontsize=12, fontweight='bold')
    ax.set_title(f'H2: Vagueness × Hardware Interaction ({n_bins_interact} bins)\n(Does vagueness help more in Software?)',
                fontsize=14, fontweight='bold')
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Growth Rate by Vagueness × Hardware:")
    print("\nSoftware:")
    print(growth_sw)
    print("\nHardware:")
    print(growth_hw)
    
    print("\n💡 What to look for:")
    print("   - If Software line is steeper: Vagueness helps Software more (H2 supported)")
    print("   - If lines are parallel: No interaction effect")
    print("   - If lines cross: Complex interaction pattern")
else:
    print("❌ Cannot plot - variables not available")

---
## 6. Save All Plots

모든 플롯을 한번에 생성하고 저장합니다.

In [ ]:
# Run the eyeball test script to generate all plots
import subprocess

print("Running eyeball test plot script...")
result = subprocess.run(['python', 'scripts/eyeball_test_plots.py'],
                       capture_output=True, text=True)

print(result.stdout)
if result.returncode != 0:
    print("\n❌ Error:")
    print(result.stderr)
else:
    print("\n✅ All plots saved to outputs/all/eyeball_test/")

---
## Summary

이 notebook을 통해 확인한 내용:

✅ **변수 분포 (Distributions)**
- 연속형 변수: z_vagueness, early_funding_musd, z_employees_log, z_firm_age
- 범주형 변수: growth, is_hardware, founder_serial

✅ **H1 관계 (Early Funding)**
- Early Funding ~ Vagueness
- Early Funding by Architecture (SW vs HW)
- Early Funding ~ Controls (Employees, Firm Age)

✅ **H2 관계 (Growth)**
- Growth ~ Vagueness (by quintile)
- Growth by Architecture
- **Vagueness × Hardware Interaction** (핵심!)

### Next Steps

1. **문제 발견 시**: 변수 분포나 관계가 이상하면 데이터 전처리 재검토
2. **정상이면**: 모델 추정으로 진행
   ```bash
   python -m src.cli run-models --dataset all
   python -m src.cli generate-plots --dataset all
   ```

---